# Big Market Sales Prediction

# Importing all essentials

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
sns.set_style('whitegrid')
warnings.filterwarnings('ignore')

# Data Analysis and Collection

In [ ]:
df = pd.read_csv('../input/bigmart-sales-data/Train.csv')
df.head()

In [ ]:
df.shape

In [ ]:
df.info()

Categorical Features:
- Item_Identifier
- Item_Fat_Content
- Item_Type
- Outlet_Indentifier
- Outlet_Size
- Outlet_Location-Type
- Outlet_Type

In [ ]:
# checking for missing values
df.isnull().sum()

Handling Missing Values

- MEAN -> AVERAGE
- MODE -> MORE REPEATED VALUE

In [ ]:
df['Item_Weight'].mean()

In [ ]:
# filling missing values in "Item-Weight" with MEAN VALUE
df['Item_Weight'].fillna(df['Item_Weight'].mean(), inplace=True)

In [ ]:
# Mode of "Outlet_Size" column
df['Outlet_Size'].mode()

In [ ]:
# filling the missing values in "Outlet_Size" column with Mode
mode_of_Outlet_size = df.pivot_table(values='Outlet_Size', columns='Outlet_Type', aggfunc=(lambda x: x.mode()[0]))

In [ ]:
print(mode_of_Outlet_size)

In [ ]:
miss_values = df['Outlet_Size'].isnull()

In [ ]:
print(miss_values)

In [ ]:
df.loc[miss_values, 'Outlet_Size'] = df.loc[miss_values, 'Outlet_Type'].apply(lambda x:mode_of_Outlet_size[x])

In [ ]:
#checking for missing values
df.isnull().sum()

# Data Analysis

In [ ]:
df.describe()

In [ ]:
# Item_Weight distribution
plt.figure(figsize=(6,6))
sns.distplot(df['Item_Weight'])
plt.show()

In [ ]:
# Item Visibility distribution
plt.figure(figsize=(6,6))
sns.distplot(df['Item_Visibility'])
plt.show()

In [ ]:
# Item MRP distribution
plt.figure(figsize=(6,6))
sns.distplot(df['Item_MRP'])
plt.show()

In [ ]:
# Item_Outlet_Sales distribution
plt.figure(figsize=(6,6))
sns.distplot(df['Item_Outlet_Sales'])
plt.show()

In [ ]:
# Outlet_Establishment_Year column
plt.figure(figsize=(6,6))
sns.countplot(x='Outlet_Establishment_Year', data=df)
plt.show()

In [ ]:
# Item_Fat_Content column
plt.figure(figsize=(6,6))
sns.countplot(x='Item_Fat_Content', data=df)
plt.show()

In [ ]:
# Item_Type column
plt.figure(figsize=(30,6))
sns.countplot(x='Item_Type', data=df)
plt.show()

In [ ]:
# Outlet_Size column
plt.figure(figsize=(6,6))
sns.countplot(x='Outlet_Size', data=df)
plt.show()

# Data Pre=Processing

In [ ]:
df.head()

In [ ]:
df['Item_Fat_Content'].value_counts()

In [ ]:
df.replace({'Item_Fat_Content':{'low fat':'Low Fat', 'LF':'Low Fat', 'reg':'Regular'}},inplace=True)

In [ ]:
df['Item_Fat_Content'].value_counts()

### Label Encoding

In [ ]:
from sklearn.preprocessing import LabelEncoder

In [ ]:
encoder = LabelEncoder()

In [ ]:
df['Item_Identifier'] = encoder.fit_transform(df['Item_Identifier'])

df['Item_Fat_Content'] = encoder.fit_transform(df['Item_Fat_Content'])

df['Item_Type'] = encoder.fit_transform(df['Item_Type'])

df['Outlet_Identifier'] = encoder.fit_transform(df['Outlet_Identifier'])

df['Outlet_Size'] = encoder.fit_transform(df['Outlet_Size'])

df['Outlet_Location_Type'] = encoder.fit_transform(df['Outlet_Location_Type'])

df['Outlet_Type'] = encoder.fit_transform(df['Outlet_Type'])

In [ ]:
df.head()

# Splitting features and Target

In [ ]:
X = df.iloc[:,:-1].values
y = df.iloc[:,-1].values

In [ ]:
print(X)

In [ ]:
print(y)

# Splitting into Train Test Split

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.2, random_state=0)

In [ ]:
print(X.shape, X_train.shape, X_test.shape)

# Training Algortihm

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

In [ ]:
model = Sequential()

model.add(Dense(20, activation='relu'))
model.add(Dense(10, activation='relu'))
model.add(Dense(5, activation='relu'))

model.add(Dense(1)) # output layer

model.compile(optimizer='rmsprop', loss='mse')

In [ ]:
model.fit(x = X_train, y = y_train, epochs=50)

In [ ]:
model.summary()

In [ ]:
loss_df = pd.DataFrame(model.history.history)

In [ ]:
loss_df

In [ ]:
loss_df.plot()

From above graph we can say that loss of our model decreasing significantly.

# Model Evaluation

In [ ]:
# Model evaluation on test data
test_eval = model.evaluate(X_test, y_test, verbose=0)
print(test_eval)

In [ ]:
# model evaluation on train set
train_eval = model.evaluate(X_train, y_train, verbose=0)
print(train_eval)

In [ ]:
# Checking difference between train_eval and test_eval
model_diff = train_eval - test_eval
print(model_diff)

In [ ]:
# Train prediction on test data
train_prediction = model.predict(X_train)

In [ ]:
# Test prediction on test data
test_prediction = model.predict(X_test)

In [ ]:
print(train_prediction)

In [ ]:
print(test_prediction)

In [ ]:
# R Squared : R-squared measures the strength of the relationship between your model and the dependent variable on a convenient 0 – 100% scale.
from sklearn.metrics import r2_score
r2_train = r2_score(y_train, train_prediction)

In [ ]:
print('R squared value of train data:', r2_train)

In [ ]:
r2_test = r2_score(y_test, test_prediction)

In [ ]:
print('R squared value of test data:',r2_test)